In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import json
from pprint import pprint
from pandas.io.json import json_normalize

#Sample json
with open('../data/json/72572.json') as data_file:    
    data = json.load(data_file)
#pprint(data)

In [3]:
#level 1 data
result = json_normalize(data)
result

,HotelInfo.Address,HotelInfo.HotelID,HotelInfo.HotelURL,HotelInfo.ImgURL,HotelInfo.Name,HotelInfo.Price,Reviews
0,"<address class=""addressReset""> <span rel=""v:ad...",72572,/ShowUserReviews-g60878-d72572-Reviews-BEST_WE...,http://media-cdn.tripadvisor.com/media/Provide...,BEST WESTERN PLUS Pioneer Square Hotel,$117 - $189*,"[{'Ratings': {'Service': '4', 'Cleanliness': '..."


In [4]:
#drop columns
result = result.drop(['Reviews','HotelInfo.HotelURL','HotelInfo.ImgURL'], axis=1)
result

,HotelInfo.Address,HotelInfo.HotelID,HotelInfo.Name,HotelInfo.Price
0,"<address class=""addressReset""> <span rel=""v:ad...",72572,BEST WESTERN PLUS Pioneer Square Hotel,$117 - $189*


In [5]:
#clean html tags
#result.at[0,'HotelInfo.Address']

In [6]:
import re

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [7]:
#html tags cleaned
address = re.sub("\s\s+", "", cleanhtml(result.at[0,'HotelInfo.Address']))
#address

In [8]:
#make new list of strings if we want to compare hotel address and author address (messed up still but closer)
address2 = [x.strip() for x in address.split(',')]
address2

['77 Yesler Way', 'Seattle', 'WA 98104-2530']

In [9]:
#clean $, *, and range
#result.at[0,'HotelInfo.Price']

In [10]:
#re.sub('[!@#$*]', '', result.at[0,'HotelInfo.Price'])

In [11]:
#one line of code to split low and high prices into 2 columns
price = [x.strip() for x in re.sub('[!@#$*]', '', result.at[0,'HotelInfo.Price']).split('-')]
price

['117', '189']

In [12]:
#create new columns for cleaned data
add = pd.DataFrame(np.column_stack([price[0], price[1], address2[0], address2[1], address2[2]]), 
             columns=['HotelInfo.Price.Low', 'HotelInfo.Price.High', 'HotelInfo.Address.Street',
                      'HotelInfo.Address.City', 'HotelInfo.Address.State'])
add

,HotelInfo.Price.Low,HotelInfo.Price.High,HotelInfo.Address.Street,HotelInfo.Address.City,HotelInfo.Address.State
0,117,189,77 Yesler Way,Seattle,WA 98104-2530


In [13]:
#drop old columns, concat
result = pd.concat([add, result], axis=1)
result = result.drop(['HotelInfo.Address','HotelInfo.Price'], axis=1)
result

,HotelInfo.Price.Low,HotelInfo.Price.High,HotelInfo.Address.Street,HotelInfo.Address.City,HotelInfo.Address.State,HotelInfo.HotelID,HotelInfo.Name
0,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel


In [14]:
#second level data
result2 = json_normalize(data["Reviews"])
result2

,Author,AuthorLocation,Content,Date,"Ratings.Business service (e.g., internet access)",Ratings.Check in / front desk,Ratings.Cleanliness,Ratings.Location,Ratings.Overall,Ratings.Rooms,Ratings.Service,Ratings.Sleep Quality,Ratings.Value,ReviewID,Title
0,gowharr32,Boston,We enjoyed the Best Western Pioneer Square. My...,"March 29, 2012",NaN,NaN,5,5,5.0,5,4,4,4,UR126946257,“Excellent Hotel & Location”
1,Nancy W,"Madison, Wisconsin",Great visit to Seattle thanks to our stay at t...,"March 27, 2012",NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,UR126795011,“Great Visit to Seattle!”
2,Janet H,"Ketchikan, Alaska","Great Location,short walk from Amtrak station,...","March 27, 2012",NaN,NaN,5,5,5.0,5,5,5,5,UR126715331,“Excellent in Everyway”
3,TimothyFlorida,Florida,Accommodation in Seattle can be expensive. Thi...,"March 24, 2012",NaN,NaN,5,5,5.0,4,5,4,5,UR126585393,"“Great hotel, location & price.”"
4,KarenArmstrong_BC,"Armstrong, BC",Very cool old building in a great location. Ch...,"March 13, 2012",NaN,NaN,4,5,4.0,3,3,3,3,UR126067021,“Cool place”
5,Shane33333,,I purchased a 7 day advanced room (actually 2 ...,"March 10, 2012",NaN,NaN,1,1,1.0,1,1,NaN,1,UR125893357,“Very bad customer service!”
6,Bnkruzn,,We stayed 3 nights and it was great. The room ...,"February 28, 2012",NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,UR125344427,“An amazing historic hotel”
7,Teacherbear,"Saskatchewan, Canada",said: This is a great little hotel located wit...,"February 23, 2012",NaN,NaN,5,5,5.0,4,5,NaN,5,UR125099396,“Super hotel when using Amtrak”
8,CandyGnomad,"Kingston, Canada",We went to Seattle on a spur of the moment ide...,"February 17, 2012",NaN,NaN,4,4,4.0,4,4,4,5,UR124738305,“Good price for a good location”
9,idahosandy,"Boise, Idaho, USA","Great little hotel. Super bedding, quiet, clos...","February 16, 2012",NaN,NaN,5,5,5.0,5,5,5,4,UR124685478,“Just passing thru”


In [15]:
#len(result2.index)

In [16]:
#multiply 1st level data to 2nd level data
result = pd.concat([result]*len(result2.index))
result.index = range(len(result2.index))
result

,HotelInfo.Price.Low,HotelInfo.Price.High,HotelInfo.Address.Street,HotelInfo.Address.City,HotelInfo.Address.State,HotelInfo.HotelID,HotelInfo.Name
0,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel
1,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel
2,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel
3,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel
4,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel
5,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel
6,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel
7,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel
8,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel
9,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel


In [17]:
#concat
result3 = pd.concat([result, result2], axis=1)
result3

,HotelInfo.Price.Low,HotelInfo.Price.High,HotelInfo.Address.Street,HotelInfo.Address.City,HotelInfo.Address.State,HotelInfo.HotelID,HotelInfo.Name,Author,AuthorLocation,Content,...,Ratings.Check in / front desk,Ratings.Cleanliness,Ratings.Location,Ratings.Overall,Ratings.Rooms,Ratings.Service,Ratings.Sleep Quality,Ratings.Value,ReviewID,Title
0,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel,gowharr32,Boston,We enjoyed the Best Western Pioneer Square. My...,...,NaN,5,5,5.0,5,4,4,4,UR126946257,“Excellent Hotel & Location”
1,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel,Nancy W,"Madison, Wisconsin",Great visit to Seattle thanks to our stay at t...,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,UR126795011,“Great Visit to Seattle!”
2,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel,Janet H,"Ketchikan, Alaska","Great Location,short walk from Amtrak station,...",...,NaN,5,5,5.0,5,5,5,5,UR126715331,“Excellent in Everyway”
3,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel,TimothyFlorida,Florida,Accommodation in Seattle can be expensive. Thi...,...,NaN,5,5,5.0,4,5,4,5,UR126585393,"“Great hotel, location & price.”"
4,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel,KarenArmstrong_BC,"Armstrong, BC",Very cool old building in a great location. Ch...,...,NaN,4,5,4.0,3,3,3,3,UR126067021,“Cool place”
5,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel,Shane33333,,I purchased a 7 day advanced room (actually 2 ...,...,NaN,1,1,1.0,1,1,NaN,1,UR125893357,“Very bad customer service!”
6,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel,Bnkruzn,,We stayed 3 nights and it was great. The room ...,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,UR125344427,“An amazing historic hotel”
7,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel,Teacherbear,"Saskatchewan, Canada",said: This is a great little hotel located wit...,...,NaN,5,5,5.0,4,5,NaN,5,UR125099396,“Super hotel when using Amtrak”
8,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel,CandyGnomad,"Kingston, Canada",We went to Seattle on a spur of the moment ide...,...,NaN,4,4,4.0,4,4,4,5,UR124738305,“Good price for a good location”
9,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel,idahosandy,"Boise, Idaho, USA","Great little hotel. Super bedding, quiet, clos...",...,NaN,5,5,5.0,5,5,5,4,UR124685478,“Just passing thru”


In [18]:
#no comments, merge by ReviewID
result4 = result3.drop(['Content','Title'], axis=1)
result4

,HotelInfo.Price.Low,HotelInfo.Price.High,HotelInfo.Address.Street,HotelInfo.Address.City,HotelInfo.Address.State,HotelInfo.HotelID,HotelInfo.Name,Author,AuthorLocation,Date,"Ratings.Business service (e.g., internet access)",Ratings.Check in / front desk,Ratings.Cleanliness,Ratings.Location,Ratings.Overall,Ratings.Rooms,Ratings.Service,Ratings.Sleep Quality,Ratings.Value,ReviewID
0,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel,gowharr32,Boston,"March 29, 2012",NaN,NaN,5,5,5.0,5,4,4,4,UR126946257
1,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel,Nancy W,"Madison, Wisconsin","March 27, 2012",NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,UR126795011
2,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel,Janet H,"Ketchikan, Alaska","March 27, 2012",NaN,NaN,5,5,5.0,5,5,5,5,UR126715331
3,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel,TimothyFlorida,Florida,"March 24, 2012",NaN,NaN,5,5,5.0,4,5,4,5,UR126585393
4,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel,KarenArmstrong_BC,"Armstrong, BC","March 13, 2012",NaN,NaN,4,5,4.0,3,3,3,3,UR126067021
5,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel,Shane33333,,"March 10, 2012",NaN,NaN,1,1,1.0,1,1,NaN,1,UR125893357
6,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel,Bnkruzn,,"February 28, 2012",NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,UR125344427
7,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel,Teacherbear,"Saskatchewan, Canada","February 23, 2012",NaN,NaN,5,5,5.0,4,5,NaN,5,UR125099396
8,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel,CandyGnomad,"Kingston, Canada","February 17, 2012",NaN,NaN,4,4,4.0,4,4,4,5,UR124738305
9,117,189,77 Yesler Way,Seattle,WA 98104-2530,72572,BEST WESTERN PLUS Pioneer Square Hotel,idahosandy,"Boise, Idaho, USA","February 16, 2012",NaN,NaN,5,5,5.0,5,5,5,4,UR124685478


In [19]:
#len(result4.ReviewID.unique())

In [20]:
#NLP comments, merge by ReviewID
result5 = result3[['ReviewID','Title','Content']]
result5

,ReviewID,Title,Content
0,UR126946257,“Excellent Hotel & Location”,We enjoyed the Best Western Pioneer Square. My...
1,UR126795011,“Great Visit to Seattle!”,Great visit to Seattle thanks to our stay at t...
2,UR126715331,“Excellent in Everyway”,"Great Location,short walk from Amtrak station,..."
3,UR126585393,"“Great hotel, location & price.”",Accommodation in Seattle can be expensive. Thi...
4,UR126067021,“Cool place”,Very cool old building in a great location. Ch...
5,UR125893357,“Very bad customer service!”,I purchased a 7 day advanced room (actually 2 ...
6,UR125344427,“An amazing historic hotel”,We stayed 3 nights and it was great. The room ...
7,UR125099396,“Super hotel when using Amtrak”,said: This is a great little hotel located wit...
8,UR124738305,“Good price for a good location”,We went to Seattle on a spur of the moment ide...
9,UR124685478,“Just passing thru”,"Great little hotel. Super bedding, quiet, clos..."
